<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<img src="mioti.png" style="height: 100px">
<center style="color:#888">Data Science with Python</center>

# DSPy7 Challenge. Scrapy

Vamos a intentar extraer datos de una web de una librería, que no es más que una web preparada para testear scrapping contra ella. 

<img src="books_to_scrape.png" style="height: 500px">

Empezaremos concretamente en esta url: http://books.toscrape.com/catalogue/page-1.html

**Inicialización**

In [1]:
import logging
import re
import pandas as pd
import json
import scrapy
from scrapy.crawler import CrawlerProcess
# Para ir viendo la salida de los comandos a medida que ocurre
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Esqueleto de los web scrapers**

In [2]:
class SimpleBookSpider(scrapy.Spider):
    name = "simplebooks"
    start_urls = [
        'http://books.toscrape.com/catalogue/page-1.html',
    ]
    
    def parse(self, response):
        # VUESTRO CÓDIGO IRÁ AQUÍ
        """En este método indicamos cuáles son los elementos que nos interesan"""
        results = response.css('article.product_pod')
        for quote in results:
            yield{
                'title': quote.css('h3 a::attr(title)').extract_first(),
                'link': quote.css('h3 a::attr(href)').extract_first(),
                'imagen': quote.css('a img::attr(src)').extract_first(),
                'price': quote.css('p.price_color::text').extract_first(),
                'rating': quote.css('p.star-rating::attr(class)').extract_first().split()[-1],
                'stock' :quote.css('p.instock.availability::text').extract()
            }
         

**Ejercicio 1:** Obten en un dataframe una lista de los libros en esta página (no en toda la librería), con título y precio.

In [3]:
process = CrawlerProcess({'FEED_FORMAT': 'csv',
                          'FEED_URI': 'rating.csv'})
process.crawl(SimpleBookSpider)
process.start()


2021-04-21 11:36:41 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-04-21 11:36:41 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-04-21 11:36:41 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-04-21 11:36:41 [scrapy.crawler] INFO: Overridden settings:
{}
2021-04-21 11:36:41 [scrapy.extensions.telnet] INFO: Telnet Password: 47cb8b3daffb4675
2021-04-21 11:36:41 [py.warnings] WARNING: C:\Users\JPAS\Anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)

2021-04-2

2021-04-21 11:36:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://books.toscrape.com/catalogue/page-1.html> (referer: None)
2021-04-21 11:36:42 [scrapy.core.scraper] DEBUG: Scraped from <200 http://books.toscrape.com/catalogue/page-1.html>
{'title': 'A Light in the Attic', 'link': 'a-light-in-the-attic_1000/index.html', 'imagen': '../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg', 'price': '£51.77', 'rating': 'Three', 'stock': ['\n    ', '\n    \n        In stock\n    \n']}
2021-04-21 11:36:42 [scrapy.core.scraper] DEBUG: Scraped from <200 http://books.toscrape.com/catalogue/page-1.html>
{'title': 'Tipping the Velvet', 'link': 'tipping-the-velvet_999/index.html', 'imagen': '../media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg', 'price': '£53.74', 'rating': 'One', 'stock': ['\n    ', '\n    \n        In stock\n    \n']}
2021-04-21 11:36:42 [scrapy.core.scraper] DEBUG: Scraped from <200 http://books.toscrape.com/catalogue/page-1.html>
{'title': 'Soumission', 'link':

2021-04-21 11:36:42 [scrapy.core.engine] INFO: Spider closed (finished)


In [4]:
df = pd.read_csv('rating.csv')
df

,title,link,imagen,price,rating,stock
0,A Light in the Attic,a-light-in-the-attic_1000/index.html,../media/cache/2c/da/2cdad67c44b002e7ead0cc356...,£51.77,Three,"\n ,\n \n In stock\n \n"
1,Tipping the Velvet,tipping-the-velvet_999/index.html,../media/cache/26/0c/260c6ae16bce31c8f8c95dadd...,£53.74,One,"\n ,\n \n In stock\n \n"
2,Soumission,soumission_998/index.html,../media/cache/3e/ef/3eef99c9d9adef34639f51066...,£50.10,One,"\n ,\n \n In stock\n \n"
3,Sharp Objects,sharp-objects_997/index.html,../media/cache/32/51/3251cf3a3412f53f339e42cac...,£47.82,Four,"\n ,\n \n In stock\n \n"
4,Sapiens: A Brief History of Humankind,sapiens-a-brief-history-of-humankind_996/index...,../media/cache/be/a5/bea5697f2534a2f86a3ef27b5...,£54.23,Five,"\n ,\n \n In stock\n \n"
5,The Requiem Red,the-requiem-red_995/index.html,../media/cache/68/33/68339b4c9bc034267e1da611a...,£22.65,One,"\n ,\n \n In stock\n \n"
6,The Dirty Little Secrets of Getting Your Dream...,the-dirty-little-secrets-of-getting-your-dream...,../media/cache/92/27/92274a95b7c251fea59a2b8a7...,£33.34,Four,"\n ,\n \n In stock\n \n"
7,The Coming Woman: A Novel Based on the Life of...,the-coming-woman-a-novel-based-on-the-life-of-...,../media/cache/3d/54/3d54940e57e662c4dd1f3ff00...,£17.93,Three,"\n ,\n \n In stock\n \n"
8,The Boys in the Boat: Nine Americans and Their...,the-boys-in-the-boat-nine-americans-and-their-...,../media/cache/66/88/66883b91f6804b2323c836933...,£22.60,Four,"\n ,\n \n In stock\n \n"
9,The Black Maria,the-black-maria_991/index.html,../media/cache/58/46/5846057e28022268153beff6d...,£52.15,One,"\n ,\n \n In stock\n \n"


**Ejercicio 2:** Ahora captura también el rating de los libros.